# Working with the GitHub API

This notebook demonstrates how to use the **GitHub REST API** with Python's `requests` library: making HTTP requests, using query parameters, and handling JSON responses.

In [24]:
import requests
import json

GITHUB_ENDPOINT = "https://api.github.com"

## 1. Setup and basic request

We import `requests` and `json`, set the base API endpoint, then send a **GET** request to verify the API is reachable. The `timeout=10` prevents the script from hanging if the server doesn't respond.

We inspect the **HTTP status code** (e.g. 200 = OK) and the **Content-Type** header to confirm we're talking to the API correctly.

In [25]:
response = requests.get(GITHUB_ENDPOINT, timeout=10)

print(f"HTTP status code: {response.status_code}")
print(f"Content-Type: {response.headers.get('Content-Type')}")

HTTP status code: 200
Content-Type: application/json; charset=utf-8


## 2. Search repositories with query parameters

We call the [Search Repositories](https://docs.github.com/en/rest/search/search#search-repositories) endpoint and pass **query parameters** in a dictionary:

- **`q`** – search query (e.g. "python devops")
- **`sort`** – sort by `stars`, `forks`, etc.
- **`order`** – `desc` or `asc`
- **`per_page`** – number of results (max 100)

`requests.get(..., params=query_params)` builds the URL with the correct query string. We use **`raise_for_status()`** to raise an exception on 4xx/5xx responses so we don't silently ignore errors.

In [26]:
search_url = f"{GITHUB_ENDPOINT}/search/repositories"

# https://docs.github.com/en/rest/search/search?apiVersion=2022-11-28#search-repositories
query_params = {
    "q": "python devops",
    "sort": "stars",
    "order": "desc",
    "per_page": 5,
}

response = requests.get(search_url, params=query_params, timeout=10)
response.raise_for_status()  # raise an exception if the request is 4xx or 5xx

print(f"Requested URL: {response.url}")
results = response.json()

Requested URL: https://api.github.com/search/repositories?q=python+devops&sort=stars&order=desc&per_page=5


## 3. Display results

The response body is JSON. We use **`response.json()`** to parse it into a Python dict. The search response has:

- **`total_count`** – total number of matching repositories
- **`items`** – list of repository objects (each has `name`, `stargazers_count`, etc.)

We loop over `items` to print each repo name and star count, then pretty-print the first repository's full JSON for debugging.

In [27]:
print(f"Found {results.get('total_count')} repositories")

for repo in results.get('items', []):
    print(f"  - {repo.get('name')} (Stars: {repo.get('stargazers_count')})")

# Print the first repository (useful for debugging)
print(json.dumps(results.get('items', [])[0], indent=2))

Found 9711 repositories
  - devops-exercises (Stars: 81144)
  - sentry (Stars: 43162)
  - cli (Stars: 37570)
  - jumpserver (Stars: 29863)
  - DevOps-Roadmap (Stars: 18741)
{
  "id": 212639071,
  "node_id": "MDEwOlJlcG9zaXRvcnkyMTI2MzkwNzE=",
  "name": "devops-exercises",
  "full_name": "bregman-arie/devops-exercises",
  "private": false,
  "owner": {
    "login": "bregman-arie",
    "id": 10349437,
    "node_id": "MDQ6VXNlcjEwMzQ5NDM3",
    "avatar_url": "https://avatars.githubusercontent.com/u/10349437?v=4",
    "gravatar_id": "",
    "url": "https://api.github.com/users/bregman-arie",
    "html_url": "https://github.com/bregman-arie",
    "followers_url": "https://api.github.com/users/bregman-arie/followers",
    "following_url": "https://api.github.com/users/bregman-arie/following{/other_user}",
    "gists_url": "https://api.github.com/users/bregman-arie/gists{/gist_id}",
    "starred_url": "https://api.github.com/users/bregman-arie/starred{/owner}{/repo}",
    "subscriptions_url":

---
# POST requests: sending data and changing state

So far we've worked with **GET** requests. **POST** requests are just as common and useful when we want to **send data to a server** and actually **cause a change of state** on the server or remote application.

- **`data`** – use for **form-encoded** bodies (e.g. traditional web forms).
- **`json`** – use for **JSON payloads**. When you pass a Python dictionary to the `json=` keyword argument, `requests` **automatically serializes** it to JSON. You don't need to call `json.dumps()` yourself, and `requests` **automatically sets** the `Content-Type: application/json` header.

Response handling is the same as with GET: use **`response.status_code`**, **`response.json()`** to parse the body, and **`response.raise_for_status()`** to raise an exception on 4xx/5xx so we get immediate, fail-fast feedback.

We'll use a **dummy endpoint** instead of the GitHub API: most GitHub POST endpoints require **authentication** because POST requests perform operations on server-side data, so auth is typically required. For a simple, no-auth example we use **httpbin.org**.